In [5]:
import ir_datasets
dataset = ir_datasets.load("msmarco-passage")
for doc in dataset.docs_iter():
    doc # namedtuple<doc_id, text>

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] If you have a local copy of https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz, you can symlink it here to avoid downloading it again: C:\Users\Dvalv\.ir_datasets\downloads\31644046b18952c1386cd4564ba2ae69
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
[INFO] [finished] https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz: [05:52] [1.06GB] [3.00MB/s]
[INFO] [starting] fixing encoding                                                                            
fixing encoding: 2.42GB [08:27, 4.77MB/s]

In [4]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)

ranked = {}

# Rank the top 1000 passages for each query
for query in dataset.queries_iter():
    query_id = query.query_id
    query_text = query.query_text
    for doc in dataset.docs_iter():
        doc_id = doc.doc_id
        doc_text = doc.text
        score = model.predict([(query_text, doc_text)])[0]
        if query_id not in ranked:
            ranked[query_id] = []
        ranked[query_id].append((doc_id, score))
    ranked[query_id] = sorted(ranked[query_id], key=lambda x: x[1], reverse=True)[:1000]

array([-11.0084095, -11.049721 , -11.087914 ], dtype=float32)